In [1]:
import numpy as np
import imageio
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA3_256
from Crypto.Signature import pkcs1_15

def image_histogram_normalization(image):
    image_max = np.max(image)
    image_min = np.min(image)
    if image_max - image_min != 0:
        image = ((((image - image_min) / (image_max - image_min)) * (2**8 - 1)).round()).astype(np.uint8)
    return image

image = image_histogram_normalization(imageio.imread('psycho.png'))

tabBits = image.ravel()
tabBits = tabBits[tabBits != 0]  #Usun zera z wejscia

tc = 0

processNumbers = []
for x in tabBits:
    tc += 1
    if tc < 4000:
        processNumbers.append(x)

#Zamiana na bajty
random_bytes = bytes(processNumbers)

#Generowanie hasza obrazu
hash_obj = SHA3_256.new(random_bytes)

#Generowanie klucza prywatnego RSA na podstawie TRNG
seed = int.from_bytes(hash_obj.digest(), byteorder='big') % (2**32)
np.random.seed(seed)

key = RSA.generate(2048, np.random.bytes)
private_pem = key.export_key(format='PEM')
with open('private_key.pem', 'wb') as f:
    f.write(private_pem)

#Generowanie klucza publicznego
public_key = key.publickey()
public_pem = public_key.export_key(format='PEM')
with open('public_key.pem', 'wb') as f:
    f.write(public_pem)

#Podpisywanie hasza obrazu kluczem prywatnym
signature = pkcs1_15.new(key).sign(hash_obj)

#Zapis podpisu do pliku
with open('image_signature.bin', 'wb') as f:
    f.write(signature)

print("Klucze RSA i podpis obrazu zapisane do plików")

Klucze RSA i podpis obrazu zapisane do plików
